In [5]:
import os, sys
from osgeo import gdal
from osgeo import gdalconst
import numpy as np
import fnmatch
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm

# To install gdal
conda install -c conda-forge gdal

In [6]:
def ReadImage(fname, readflag=1):
    src = gdal.Open(fname)
    projection = src.GetProjection()
    geotransform = src.GetGeoTransform()
    datatype = src.GetRasterBand(1).DataType
    datatype = gdal.GetDataTypeName(datatype)
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)
    cols = src.RasterXSize
    rows = src.RasterYSize
    Image = 0
    if readflag:
        Image = src.GetRasterBand(1).ReadAsArray()
        print('Image shape: %d %d' % (Image.shape))
    print('Spatial resolution: %f %f' % (xres,yres))
    return Image, projection, geotransform, (ulx,uly), (lrx,lry), src

def Setcmap(D, inv=1):
    cmap = plt.cm.jet
    cmaplist = [cmap(i) for i in range(cmap.N)]
    cmaplist[0] = (.0, .0, .0, 1.0)
    if inv:
        cmap = LinearSegmentedColormap.from_list('Custom_cmap', cmaplist, cmap.N)
    else:
        cmap = LinearSegmentedColormap.from_list('Custom_cmap', cmaplist[::-1], cmap.N)
    plt.register_cmap(name='Custom_cmap', cmap=cmap)
    bounds = np.linspace(0, D, D+1)
    norm = BoundaryNorm(bounds, cmap.N)
    return cmap, norm
    
def blockshaped(arr, nrows, ncols):
    h, w = arr.shape
    return (arr.reshape(h//nrows, nrows, -1, ncols)
               .swapaxes(1,2)
               .reshape(-1, nrows, ncols))

def unblockshaped(arr, h, w):
    n, nrows, ncols = arr.shape
    return (arr.reshape(h//nrows, -1, nrows, ncols)
               .swapaxes(1,2)
               .reshape(h, w))

# Read S2 bands

In [7]:
folder = 'S2B_MSIL1C_20181010T104019_N0206_R008_T32ULC_20181010T161145.SAFE'

In [8]:
infolder = os.path.join(folder, 'GRANULE')
infolder = os.path.join(infolder, os.listdir(infolder)[0], 'IMG_DATA')
infile = os.path.join(infolder, fnmatch.filter(os.listdir(infolder), '*B02.jp2')[0])
print('Reading '+infile)
Blue, PROJ, GEOTR, p1, p2, SRC = ReadImage(infile)
rows, cols = SRC.RasterYSize, SRC.RasterXSize
infile = os.path.join(infolder, fnmatch.filter(os.listdir(infolder), '*B03.jp2')[0])
print('Reading '+infile)
Green, _, _, _, _, _ = ReadImage(infile)
infile = os.path.join(infolder, fnmatch.filter(os.listdir(infolder), '*B04.jp2')[0])
print('Reading '+infile)
Red, _, _, _, _, _ = ReadImage(infile)
infile = os.path.join(infolder, fnmatch.filter(os.listdir(infolder), '*B08.jp2')[0])
print('Reading '+infile)
NIR, _, _, _, _, _ = ReadImage(infile)

Reading S2B_MSIL1C_20181010T104019_N0206_R008_T32ULC_20181010T161145.SAFE/GRANULE/L1C_T32ULC_A008328_20181010T104018/IMG_DATA/T32ULC_20181010T104019_B02.jp2
Image shape: 10980 10980
Spatial resolution: 10.000000 -10.000000
Reading S2B_MSIL1C_20181010T104019_N0206_R008_T32ULC_20181010T161145.SAFE/GRANULE/L1C_T32ULC_A008328_20181010T104018/IMG_DATA/T32ULC_20181010T104019_B03.jp2
Image shape: 10980 10980
Spatial resolution: 10.000000 -10.000000
Reading S2B_MSIL1C_20181010T104019_N0206_R008_T32ULC_20181010T161145.SAFE/GRANULE/L1C_T32ULC_A008328_20181010T104018/IMG_DATA/T32ULC_20181010T104019_B04.jp2
Image shape: 10980 10980
Spatial resolution: 10.000000 -10.000000
Reading S2B_MSIL1C_20181010T104019_N0206_R008_T32ULC_20181010T161145.SAFE/GRANULE/L1C_T32ULC_A008328_20181010T104018/IMG_DATA/T32ULC_20181010T104019_B08.jp2
Image shape: 10980 10980
Spatial resolution: 10.000000 -10.000000


# Generate blocks

In [9]:
window = 244
A = blockshaped(Blue, window, window)
Stack = np.expand_dims(A, axis=0)
A = blockshaped(Green, window, window)
Stack = np.concatenate((Stack, np.expand_dims(A, axis=0)), axis=0)
A = blockshaped(Red, window, window)
Stack = np.concatenate((Stack, np.expand_dims(A, axis=0)), axis=0)
A = blockshaped(NIR, window, window)
Stack = np.concatenate((Stack, np.expand_dims(A, axis=0)), axis=0)
del A, Blue, Green, Red, NIR
Stack.shape

(4, 2025, 244, 244)

# Standardize tensor

In [ ]:
X = np.float32(Stack)
X[X>10000] = 10000
X = X / 10000.
X = np.rollaxis(X, 0, 4)
X.shape

In [ ]:
X=X[:,:128,:128,:]

In [ ]:
std_params = np.load('std_params.npy')
mu = std_params[0,:]
std = std_params[1,:]

In [ ]:
for q in range(X.shape[3]):
    tmp = np.copy(X[:, :, :, q])
    tmp = tmp.flatten()
    print(mu[q], std[q])
    tmp[tmp>0] = (tmp[tmp>0] - mu[q]) / std[q]
    X[:, :, :, q] = tmp.reshape(X.shape[0], X.shape[1], X.shape[2])
    del tmp
X.shape

# Classification

In [ ]:
## U-net
from tensorflow.keras import layers
from tensorflow import keras

from keras.models import Model, load_model
from keras.layers import Input
from keras import backend as K
from keras.layers import merge, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Lambda
from keras.optimizers import Adam
from keras.layers.merge import concatenate

img_size=(128, 128, 4)
num_classes = 9

def unet(img_size, num_classes):
    inputs = keras.Input(shape=img_size )

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2,  padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model


# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = unet(img_size, num_classes)
model.summary()





In [ ]:
input_shape = (128, 128, 4)
model = unet(input_shape,9)
model.summary()
batch_size = 25
fnmodel = 'model.h5'
model.load_weights(filepath = fnmodel)

In [ ]:
Resp = model.predict(X, batch_size=3*batch_size).argmax(axis=-1)
Class = unblockshaped(Resp, 5760, 5760)

In [ ]:
cmap, norm = Setcmap(9)
im = plt.imshow(Class, cmap=cmap, norm=norm, vmin=0, vmax=9)
plt.title('Classification')
plt.set_cmap(cmap)
cb = plt.colorbar(im, fraction=0.046, pad=0.04, ticks=range(9))
cb.set_ticks(np.arange(9) + .5)
cb.set_ticklabels(np.arange(9))
plt.gcf().set_size_inches(15, 14)
plt.show()